
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
from ecolime.flat_files import *

from minime import *
from minime.util.building import build_reactions_from_genbank, add_transcription_reaction
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

## Begin by loading metabolites

In [2]:
def fix_id(id_str):
    return id_str.replace("_DASH_", "__")

In [3]:
me = MEmodel("iJO1366-ME")
me.id = "iJO1366-ME"
me.compartments = {"p": "Periplasm", "e": "Extra-organism",
                   "c": "Cytosol"}
compartment_lookup = {v: k for k, v in me.compartments.items()}

In [4]:
met_info = pandas.read_csv(
    join(ecoli_files_dir, "metabolites.txt"),
    delimiter="\t", header=None, index_col=0,
    names=["id", "name", "formula", "compartment", "data_source"])

In [5]:
for met_id in met_info.index:
    fixed_id = fix_id(met_id)
    for compartment in met_info.compartment[met_id].split("AND"):
        compartment = compartment.strip()
        if compartment == "No_Compartment":
            print "Assigned %s to e" % met_id
            compartment = me.compartments["e"]
        metab = Metabolite(fixed_id + "_" + compartment_lookup[compartment])
        metab.name = met_info.name[met_id]
    
        metab.formula = met_info.formula[met_id]
        me.add_metabolites(metab)

Assigned tl to e
Assigned hemed to e
Assigned pqq to e
Assigned dpm to e
Assigned 23bpg to e
Assigned tqn to e


Add generic ions to simplify complexes

In [6]:
divalent_list = divalent_list
monovalent_list = monovalent_list
comp = "_c"  # compartment
# Divalent
div = cobra.Metabolite('generic_divalent' + comp)
div.name = 'Generic divalent ion'
me.add_metabolites(div)
for ion in divalent_list:
    rxn1 = cobra.Reaction(ion + comp + '_to_generic')
    ion_dict = {}
    met1 = me.metabolites.get_by_id(ion + comp)
    ion_dict[met1] = -1
    ion_dict[div] = 1
    rxn1.add_metabolites(ion_dict)
    me.add_reaction(rxn1)
# Monovalent
mono = cobra.Metabolite('generic_monovalent' + comp)
mono.name = 'Generic monovalent ion'
for ion in monovalent_list:
    rxn2 = cobra.Reaction(ion + comp + '_to_generic')
    ion_dict = {}
    met2 = me.metabolites.get_by_id(ion + comp)
    ion_dict[met2] = -1
    ion_dict[mono] = 1
    rxn2.add_metabolites(ion_dict)
    me.add_reaction(rxn2)

## Build Metabolic reactions

In [7]:
reaction_info = get_reaction_info_frame()
reaction_dict = get_reaction_matrix_dict()

In [8]:
for r_id in reaction_info.index:
    reaction = StoichiometricData(r_id, me)
    reaction._stoichiometry = {fix_id(k): v
                               for k, v in reaction_dict[r_id].items()}
    reaction.lower_bound = \
        -1000. if reaction_info.is_reversible[r_id] else 0.
    reaction.upper_bound = 1000.

Also make a dummy reaction

In [9]:
dummy = StoichiometricData("dummy_reaction", me)
dummy.lower_bound = 0
dummy.upper_bound = 1000
dummy._stoichiometry = {}

Boundary Reactions

In [10]:
sources_sinks = pandas.read_csv(join(ecoli_files_dir, "reaction_matrix_sources_and_sinks.txt"), delimiter="\t",
                                header=None, names=["rxn_id", "met_id", "compartment", "stoic"], index_col=1)
sources_sinks.index = [fix_id(i) for i in sources_sinks.index]

source_amounts = pandas.read_csv(join(ecoli_files_dir, "exchange_bounds.txt"),
                                 delimiter="\t", index_col=0, names=["met_id", "amount"])
source_amounts.index = [fix_id(i) for i in source_amounts.index]


for met_id in sources_sinks.index:
    model_met_id = met_id + "_" + compartment_lookup[sources_sinks.compartment[met_id]]
    # EX_ or DM_ + met_id
    reaction_id = sources_sinks.rxn_id[met_id][:3] + model_met_id
    reaction = cobra.Reaction(reaction_id)
    me.add_reaction(reaction)
    reaction.add_metabolites({me.metabolites.get_by_id(model_met_id): -1})
    # set bounds on exchanges
    if reaction.id.startswith("EX_") and met_id in source_amounts.index:
        reaction.lower_bound = -source_amounts.amount[met_id]

Set bounds on boundary reactions

In [11]:
# Add all RNA related metabolites/reactions. Important to run first if using TUs!
RNA_pos_dict = build_reactions_from_genbank(me, join(ecoli_files_dir, "NC_000913.2.gb"), using_TUs=True)

TODO deal with selenocystine


/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:217 UserWarning: ribosome not found


## Build ribosome and RNA Polymerase

In [12]:
# Add generic_16s,23s, and 5s rRNA formation reactions

list_16s = ['b3851', 'b3968', 'b3756', 'b3278', 'b4007', 'b2591', 'b0201']
list_23s = ['b3854', 'b3970', 'b3758', 'b3275', 'b4009', 'b2589', 'b0204']
list_5s = ['b3855', 'b3971', 'b3759', 'b3274', 'b4010', 'b2588', 'b0205', 'b3272']
list_RNase = ['RNase_T_dim_mod_4:mg2', 'RNase_BN_dim_mod_2:zn2', 'Rnd_mono_mod_5:mg2',
             'Rnb_mono_mod_1:mg2', 'Rph_mono_mod_mg2']
list_RNase = [R.replace('mg2', 'generic_divalent').replace('zn2', 'generic_divalent') 
             for R in list_RNase]
for i in ['16s', '23s', '5s', 'RNase']:
    for RNA in eval('list_' + i):
        if RNA.startswith('b'):
            RNA_id = 'RNA_' + RNA
            r = cobra.Reaction("rRNA_" + RNA + '_to_generic')
        else:
            RNA_id = RNA
            r = cobra.Reaction("spliceosome_" + RNA + '_to_generic')
        me.add_reaction(r)
        r.reaction = RNA_id + ' <=> generic_' + i

unknown metabolite 'generic_16s' created
unknown metabolite 'generic_23s' created
unknown metabolite 'generic_5s' created
unknown metabolite 'RNase_T_dim_mod_4:generic_divalent' created
unknown metabolite 'generic_RNase' created
unknown metabolite 'RNase_BN_dim_mod_2:generic_divalent' created
unknown metabolite 'Rnd_mono_mod_5:generic_divalent' created
unknown metabolite 'Rnb_mono_mod_1:generic_divalent' created
unknown metabolite 'Rph_mono_mod_generic_divalent' created


In [13]:
ribosome_complex = ComplexData("ribosome", me)
ribosome_components = ribosome_complex.stoichiometry
ribosome_modifications = ribosome_complex.modifications

# 30S assembly associated reactions
# Phase 1
# ribosome 30s assembly via 2 GTP bind to ribosome 30s asembly factor
mod_1 = ModificationData('gtp_bound_30S_assembly_factor_phase1', me)
mod_1.stoichiometry = {'gtp_c': 2, 'h2o_c': 2, 'h_c': -2, 'pi_c': -2}
mod_1.enzyme = 'Era_dim'
ribosome_modifications[mod_1.id] = -1

mod_2 = ModificationData('RbfA_mono_aasembly_factor_phase1', me)
mod_2.enzyme = 'RbfA_mono'
ribosome_modifications[mod_2.id] = -1

mod_3 = ModificationData('RimM_mono_aasembly_factor_phase1', me)
mod_3.enzyme = 'RimM_mono'
ribosome_modifications[mod_3.id] = -1

ribosome_components['generic_16s'] = 1
# 30S Listed as [rpsA -rpsU], sra 
# [rplA-rplF],  rplI, [rplK-rplY],
# [rpmA-rpmJ]

# TODO change these to complex names
for i in ["b0911", "b0169", "b3314", "b3296", "b3303", "b4200", "b3341",
          "b3306", "b3230", "b3321", "b3297", "b3342", "b3298", "b3307", 
          "b3165", "b2609", "b3311", "b4202", "b3316", "b0023", "b3065",
          "b1480"]:
    ribosome_components["protein_" + i] = 1
# Phase 2
ribosome_components['mg2_c'] = 60


# 50s reactions
ribosome_components['generic_23s'] = 1
ribosome_components['generic_5s'] = 1

# 50S listed as [rplA-rplF],rplJ, rplI, rplK [rplM-rplY], [rpmA-rpmJ]    
for i in ["b3984", "b3317", "b3320", "b3319", "b3308", "b3305", 
          "b3958", "b4203", "b3983", "b3231", "b3310", "b3301",
          "b3313", "b3294", "b3304", "b2606", "b1716", "b3186",
          "b3315", "b3318", "b3309", "b2185", 
          "b3185", "b3637", "b3312", "b3302", "b3936", "b1089", 
          "b3636", "b3703", "b1717", "b3299"]:
    ribosome_components["protein_" + i] = 1
# [rplJ, 2(2[rpIL7])]
ribosome_components["protein_" + "b3986"] = 4

ribosome_components['mg2_c'] += 111
# get ribosome ready for translation
# ribosome_50 + ribosome_30 + trigger_factor -> rib_70
ribosome_components['Tig_mono'] = 1

# rib_70 + If_1 + If_3 -> rib_50_trigger_factor + rib30_if1_if3
ribosome_components['InfA_mono'] = 1
ribosome_components['InfC_mono'] = 1

# 1 b3168_assumedMonomer_gtp (InfB_mono) + 1 rib_30_IF1_IF3 --> 1 rib_30_ini 
ribosome_components['InfB_mono'] = 1
ribosome_components['gtp_c'] = 1 

# rib_30_ini + rib_50_trigger_factor -> ribsome_complex
ribosome_complex.create_complex_formation()

Created <Metabolite InfA_mono at 0x7fa1fa709810> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>
Created <Metabolite RimM_mono at 0x7fa1fa709a10> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>
Created <Metabolite RbfA_mono at 0x7fa1fa7099d0> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>
Created <Metabolite InfC_mono at 0x7fa1fa709a50> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>
Created <Metabolite InfB_mono at 0x7fa1fa709a90> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>
Created <Metabolite Tig_mono at 0x7fa1fa709ad0> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>
Created <Metabolite Era_dim at 0x7fa1fa709b90> in <ComplexFormation formation_ribosome at 0x7fa1fa709950>


In [14]:
RNAP_complex = ComplexData("RNA_Polymerase", me)
RNAP_components = RNAP_complex.stoichiometry
# Core RNA Polymerase Enzyme
for i in {"b3295" : "rpoA", "b3988" : "rpoC", "b3987" : "rpoB"}:
    if i == "b3295":
        RNAP_components["protein_" + i] = 2
    else:
        RNAP_components["protein_" + i] = 1

RNAP_complex.create_complex_formation()

In [15]:
import itertools

In [16]:
def find_existing_genome_region(me_model,left_pos, right_pos, TU_strand, has_5prime_triphosphate):
    try:
        TU_name = RNA_pos_dict[str(left_pos)+','+str(right_pos)]
    except:                 
        TU_name = 'excised_TU_%s_%i_%i_%s' % (TU_strand.replace('-','M').replace('+','P'), 
                                                  left_pos, right_pos, has_5prime_triphosphate)
        try:
            me.metabolites.get_by_id(TU_name)
        except:
            #print 'Creating new transcribed gene: ', TU_name
            new_TU = TranscribedGene(TU_name)
            new_TU.left_pos = left_pos
            new_TU.right_pos = right_pos
            new_TU.strand = TU_strand
            me_model.add_metabolites([new_TU])
            r = cobra.Reaction('DM_' + TU_name)
            me_model.add_reaction(r)
            r.reaction = TU_name + ' --> '

    return TU_name

In [17]:
def transcribe_all_TU_combos(me_model, TU, TU_left, TU_right, TU_strand, bnum_set, TU_seq, TU_pieces):
    
    # Create list of all left strand positions of tRNA, rRNA, sRNA that will be excised
    all_lefts = []
    for bnum in bnum_set:
        all_lefts.append(me.metabolites.get_by_id('RNA_'+bnum).left_pos)
    if len(all_lefts) > 5:
        print all_lefts
    # Create list [0:number_of_possible_excised_portions]
    num_in_combo = range(len(all_lefts))
    num_in_combo.append(len(all_lefts))
    
    for i in num_in_combo:
        
        # Iterate through all possible excised combinations that
        # Create the number of pieces in i
        for left_combo in itertools.combinations(all_lefts, i):
            left_combo_list = list(left_combo)
   
            if TU not in TU_pieces:
                TU_pieces[TU] = []
            
            # Do nothing if TU has not tRNA, rRNA to excise
            if len(left_combo_list) == 0:
                #print TU, '<---- check'
                TU_pieces[TU].append([])
                continue
                
            left_combo_list.sort() # ascending by default
            excised_TU_portions = []
            excised_TU_portion_count = 1
            
            # Look at remaining RNA portion between TU left_pos
            # and first tRNA, rRNA
            if left_combo_list[0] > TU_left:
                if TU_strand == '-':
                    has_5prime_triphosphate = 'False'
                # (+) strain has not been sliced on the left side yet, preserving
                # triphosphate group
                else:
                    has_5prime_triphosphate = 'True'
                
                
                # Add TranscribedGene for this segment if not already in model
                TU_id = find_existing_genome_region(me_model, TU_left, left_combo_list[0]-1, 
                                               TU_strand, has_5prime_triphosphate)
                
                excised_TU_portions.append(TU_id)
                excised_TU_portion_count += 1
                
            # iterate rest of possible segments
            while len(left_combo_list) > 0:
                left = left_combo_list[0]
                
                # Check to see which rRNA, tRNA this segment represents
                for bnum in bnum_set:
                    if me.metabolites.get_by_id('RNA_'+bnum).left_pos == left:
                        right = me.metabolites.get_by_id('RNA_'+bnum).right_pos
                        excised_portion_name = 'RNA_'+bnum
                
                # tRNA, rRNA loses 5' triphosphate if cleaved
                if left != TU_left and TU_strand == '+':
                    me.metabolites.get_by_id('RNA_'+bnum).has_5prime_triphosphate = 'False'
                if right != TU_right and TU_strand == '-':
                    me.metabolites.get_by_id('RNA_'+bnum).has_5prime_triphosphate = 'False'
                
                
                excised_TU_portions.append(excised_portion_name)
                excised_TU_portion_count += 1
                left_combo_list.pop(left_combo_list.index(left))
                
                # Deal with the last (far right) segment    
                if len(left_combo_list) == 0:
                    if TU_right - right > 0:
                        # (-) strain has not been sliced on the right side yet, preserving
                        # triphosphate group
                        if TU_strand == '-':
                            has_5prime_triphosphate = 'True'
                        else:
                            has_5prime_triphosphate = 'False'
                            
                        TU_id = find_existing_genome_region(me_model, right + 1, TU_right, 
                                               TU_strand, has_5prime_triphosphate)

                        excised_TU_portions.append(TU_id)
                        excised_TU_portion_count += 1
                        
                # Add in excised TU for next segment, if not tRNA, rRNA
                else:
                    next_left = left_combo_list[0]
                    TU_id = find_existing_genome_region(me_model, right + 1, next_left-1, 
                                                   TU_strand, 'False')
                    excised_TU_portion_count += 1
                    excised_TU_portions.append(TU_id)
                    
            if len(excised_TU_portions) > 0:
                TU_pieces[TU].append(excised_TU_portions)
                

In [18]:
from Bio import SeqIO


#stable_RNA_df = pandas.read_csv(join(ecoli_files_dir,'stable_RNA.txt'), delimiter='\t', index_col=0)
#stable_RNA_dict = stable_RNA_df.T.to_dict()
#TU_gene_dict = TU_gene_df.T.to_dict()
gb_file = SeqIO.read(join(ecoli_files_dir,'NC_000913.2.gb'), 'gb')                                      
full_seq = str(gb_file.seq)                                                  
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)            


TU_pieces = {}
for index, TU in TU_df.iterrows():
    seq = full_seq[TU.start:TU.stop]                                          
    if TU.strand == '-':
        seq = util.dogma.reverse_transcribe(seq) 
    index_list = index.split('_')[0:2]
    new_index = index_list[0] + '_' + index_list[1]
    new_index = new_index.replace('_from', '').replace('_with','')

    excise = False
    loci = []
    for gene in me.metabolites.query('RNA_b'):
        if gene.strand == TU.strand and gene.left_pos + 1 >= TU.start \
        and gene.right_pos <= TU.stop:
            loci.append(gene.id.replace('RNA_',''))
    loci = set(loci)
    print index, loci
    for locus in loci:
        try:
            if me.metabolites.get_by_id('RNA_'+locus).RNA_type == 'tRNA' or \
            me.metabolites.get_by_id('RNA_'+locus).RNA_type == 'rRNA' or locus == 'b3123':
                excise = True
        except:
            pass

    if excise==False:
        add_transcription_reaction(me, index, loci, seq)
    else:
        TU_data = TranscriptionData(index, me)
        TU_data.nucleotide_sequence = seq
        TU_data.RNA_products = loci
        # Must splice TU to form tRNA, rRNA, sRNA (not supported)
        transcribe_all_TU_combos(me, index, TU.start, TU.stop, TU.strand, loci, seq, TU_pieces)

TU0_13350_from_RpoD_mono set(['b1623'])
TU0_12877_from_RpoD_mono set(['b0288'])
TU0_7622_from_RpoD_mono set(['b0515', 'b0517', 'b0516'])
TU00158_from_RpoD_mono set(['b1109'])
TU0_8340_from_EG11355_MONOMER set(['b1937'])
TU0_8834_from_RpoD_mono set(['b1370'])
TU0_8688_from_RpoD_mono set(['b1178'])
TU0_1002_from_RpoD_mono set(['b1748', 'b1747', 'b1746', 'b1745', 'b1744'])
TU0_4702_with_TERM0223_from_RpoD_mono set(['b0342', 'b0343', 'b0344'])
TU0_4702_with_TERM0222_from_RpoD_mono set(['b0342', 'b0343', 'b0344'])
TU0_6513_from_RpoD_mono set(['b2011'])
TU782_from_RPON_MONOMER set(['b4002'])
TU0_3421_from_RPOE_MONOMER set(['b2378'])
TU0_13236_from_RpoD_mono set(['b1372', 'b1373'])
TU0_8597_from_RpoD_mono set(['b3684'])
TU0_14474_from_RPOH_MONOMER set(['b4258'])
TU0_12820_from_RpoD_mono set(['b0149'])
TU0_14822_from_RpoD_mono set(['b1676'])
TU0_14253_with_TERM0_1039_from_RpoD_mono set(['b0029', 'b0028', 'b0027'])
TU0_8529_from_RpoD_mono set(['b0423'])
TU00242_from_RpoD_mono set(['b2714'])
TU0

In [19]:
# Combine excision machinery into one set
rRNA_containing = ['RNase_E_tetra_mod_2:zn2', 'RNase_P_cplx_mod_2:mg2', 'generic_RNase', 'RNase_m5', 'RNase_m16', 
                   'RNase_m23', 'RNase_III_dim_mod_2:mg2', 'RNase_G_dim', 'RNase_T_dim_mod_4:mg2']

monocistronic = ['RNase_E_tetra_mod_2:zn2', 'RNase_P_cplx_mod_2:mg2', 'generic_RNase']

polycistronic_wout_rRNA = ['RNase_E_tetra_mod_2:zn2', 'RNase_P_cplx_mod_2:mg2', 'generic_RNase', 'RNase_III_dim', 
                           'RNase_G_dim', 'RNase_T_dim_mod_4:mg2']

excision_types = ['rRNA_containing', 'monocistronic', 'polycistronic_wout_rRNA']
for excision in excision_types:
    excision_dict = {}
    r = cobra.Reaction('combine_' + excision + '_excision_machinery')
    for machine in eval(excision):
        for ion in divalent_list:
            machine = machine.replace(ion,'generic_divalent')
        excision_dict[Metabolite(machine)] = -1
    excision_dict[Metabolite(excision + '_excision_set')] = 1
    r.add_metabolites(excision_dict)
    me.add_reaction(r)
                                                                                
    # Add modification data objects for TU excision reactions
    rRNA_mod = ModificationData(excision + '_excision', me)
    rRNA_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
    rRNA_mod.enzyme = excision + '_excision_set'

In [20]:
for TU, combos_of_pieces in TU_pieces.iteritems():
    for i, pieces in enumerate(combos_of_pieces):
        tRNA_count = 0
        rRNA_count = 0
        sRNA_count = 0 # not supported
        
        transcription = TranscriptionReaction('transcription_' + TU + '_slice_' + str(i+1))
        transcription_data = TranscriptionData(TU + '_slice_' + str(i+1), me)
        full_TU_data = me.transcription_data.get_by_id(TU)
        transcription_data.nucleotide_sequence = full_TU_data.nucleotide_sequence

        if len(pieces) < 2: # no splicing to do
            continue
        RNA_products = set()
        
        for piece in pieces:
            RNA_object = me.metabolites.get_by_id(piece)
            if RNA_object.RNA_type == 'tRNA':
                tRNA_count += 1
            elif RNA_object.RNA_type == 'rRNA':
                rRNA_count += 1
            else:
                sRNA_count += 1
            RNA_products.add(piece)
            
        if rRNA_count > 0:
            transcription_data.modifications['rRNA_containing_excision'] = len(pieces)-1
        elif tRNA_count == 1 and rRNA_count == 0:
            transcription_data.modifications['monocistronic_excision'] = len(pieces)-1
        elif tRNA_count > 1 and rRNA_count == 0:
            transcription_data.modifications['polycistronic_wout_rRNA_excision'] = len(pieces)-1
        else: # only applies to rnpB
            transcription_data.modifications['monocistronic_excision'] = len(pieces)-1
        
        transcription_data.RNA_products = RNA_products
        transcription.transcription_data = transcription_data
        
        me.add_reaction(transcription)
        transcription.update()

In [21]:
for rxn in me.reactions.query('transcription'):
    print rxn.id, '--->', rxn.reaction

transcription_TU0_13350_from_RpoD_mono ---> 225 utp_c + 236 atp_c + 0.00408304780551477*mu + 0.00159647169195627 RNA_Polymerase + 266 ctp_c + 274 gtp_c --> 321.512512262 biomass + RNA_b1623 + 1000 ppi_c
transcription_TU0_12877_from_RpoD_mono ---> 84 utp_c + 79 atp_c + 80 ctp_c + 0.00134198074726709*mu + 0.000524714472181433 RNA_Polymerase + 86 gtp_c --> 105.671386262 biomass + RNA_b0288 + 328 ppi_c
transcription_TU0_7622_from_RpoD_mono ---> 804 utp_c + 768 atp_c + 0.012848751835536*mu + 0.00502386196769457 RNA_Polymerase + 829 ctp_c + 749 gtp_c --> 1007.54803026 biomass + RNA_b0516 + RNA_b0517 + RNA_b0515 + 3149 ppi_c
transcription_TU00158_from_RpoD_mono ---> 358 utp_c + 321 atp_c + 0.00569831946483929*mu + 0.00222804291075216 RNA_Polymerase + 379 ctp_c + 339 gtp_c --> 446.740519262 biomass + RNA_b1109 + 1396 ppi_c
transcription_TU0_8340_from_EG11355_MONOMER ---> 97 utp_c + 0.00160303475281449*mu + 0.000626786588350465 RNA_Polymerase + 114 ctp_c + 87 gtp_c + 95 atp_c --> 125.575819262 

In [56]:
for cplx in me.complex_data:
    for protein in cplx.stoichiometry.keys():
        try:
            if len(me.metabolites.get_by_id(protein).reactions) == 1:
                print protein
        except:
            print protein

protein_b0172
protein_b2530
protein_b3470
protein_b2741
protein_b3179
protein_b0891
protein_b3282
protein_b2559
protein_b1466
protein_b3982
protein_b0618
protein_b2517
protein_b1870
protein_b0027
protein_b2318
protein_b3740
protein_b2474
protein_b4022
protein_b3605
protein_b0784
protein_b3146
protein_b1391
protein_b3169
protein_b3461
protein_b3980
protein_b3166
protein_b2946
protein_b2140
protein_b3741
protein_b2711
protein_b3343
protein_b2960
protein_b0967
protein_b0859
protein_b3344
protein_b3084
protein_b1114
protein_b2806
protein_b0015
protein_b4180
protein_b0168
protein_b3465
protein_b3952
protein_b2828
protein_b3591
protein_b0053
protein_b1269
protein_b4142
protein_b1835
protein_b3289
protein_b1922
protein_b4143
protein_b1135
protein_b0636
protein_b0177
protein_b2512
protein_b2617
protein_b2595
protein_b2477
protein_b0051
protein_b4142
protein_b1871
protein_b3345
protein_b3344
protein_b3343
protein_b1822
protein_b3067
protein_b2573
protein_b0969
protein_b2324
protein_b2785
protei

In [55]:
me.metabolites.RNA_b0172.reactions

frozenset({<Reaction DM_b0172 at 0x7fa1fb3a6d90>,
           <TranscriptionReaction transcription_TU0_12830_from_RpoD_mono at 0x7fa1fb0eed90>})

### Add Transcription/Translation from the genbank file

Add a dummy protein in as well

In [24]:
dna_sequence = "ATG" + "TTT" * 12 + "TAT"*12+ "ACG"*12 + "GAT" *12 + "AGT"*12+ "TAA"
add_transcription_reaction(me, "dummy", {"dummy"}, dna_sequence)
me.add_metabolites(TranslatedGene("protein_" + "dummy"))
translation = TranslationReaction("translation_" + "dummy")
me.add_reaction(translation)
translation.translation_data = TranslationData("dummy", me, "RNA_dummy", "protein_dummy")
translation.translation_data.compute_sequence_from_DNA(dna_sequence)
translation.update()

complex_data = ComplexData("CPLX_dummy", me)
complex_data.stoichiometry = {}
complex_data.stoichiometry["protein_" + "dummy"] = 1
complex_data.create_complex_formation()

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [25]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [26]:
# ME_complex_dict is a dict of {'complex_id': ['bnum(count)']}
ME_complex_dict = get_complex_to_bnum_dict()
# some entries in the file need to be renamed
renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
rna_components = {"b3123"}  # component id should have RNA_ instead of protein_

for cplx, value in ME_complex_dict.iteritems():
    complex_data = ComplexData(cplx, me)
    for gene in value:
        stoichiometry = gene[6]
        bnum = gene[0:5]
        comp_id = "RNA_" + bnum if bnum in rna_components \
            else "protein_" + renames.get(bnum, bnum)
        try:
            complex_data.stoichiometry[comp_id] = float(stoichiometry)
        except:
            complex_data.stoichiometry[comp_id] = float(1)

In [27]:
me.reactions.transcription_TU00058_from_RpoD_mono.reaction

'403 utp_c + 405 atp_c + 488 ctp_c + 0.00701174743024963*mu + 0.00274159324522761 RNA_Polymerase + 423 gtp_c --> 550.109433262 biomass + RNA_b2499 + RNA_b2500 + 1718 ppi_c'

In [28]:
# {modified_complex_id: ['unmodified_complex_id', {component_id: stoic}]
modification_dict = get_protein_modification_dict(generic=True)
for mod_cplx_id, mod_complex_info in iteritems(modification_dict):
    unmod_cplx_id, mods = mod_complex_info
    unmod_cplx = me.complex_data.get_by_id(unmod_cplx_id)
    cplx = ComplexData(mod_cplx_id, me)
    cplx.stoichiometry = unmod_cplx.stoichiometry
    cplx.translocation = unmod_cplx.translocation
    cplx.chaperones = unmod_cplx.chaperones
    if len(set(mod_cplx_id.split("_mod_")[1:])) == len(mods):
        for mod_comp, mod_count in iteritems(mods):
            mod_id = "mod_" + mod_comp
            try:
                mod = me.modification_data.get_by_id(mod_id)
            except:
                mod = ModificationData(mod_id, me)
                mod.stoichiometry = {mod_comp: -1}
            cplx.modifications[mod_id] = -mod_count
    else:
        print "TODO:", mod_cplx_id

TODO: hRNAP_mod_1:generic_divalent_mod_2:generic_divalent
TODO: GltX_mono_mod_generic_divalent_mod_1:generic_divalent


Some modifications are enzyme-catalyzed

TODO: Fe-S enzyme-catalyzed modifications

In [29]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [30]:
# todo bmocogdp_c mods
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER',
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER',
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', 
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER',
                 'NAP-CPLX':'NAPD-MONOMER',
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'}

In [31]:
target_list = ['4fe4s_c', 'LI_c', '2fe2s_c', '3fe4s_c',
               'NiFeCoCN2_c',
               'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model

for met_id in target_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite '4fe4s_c' created
unknown metabolite 'LI_c' created
unknown metabolite '2fe2s_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'NiFeCoCN2_c' created


Build all complex formation reactions

TODO: This shouldn't be prining out stuff. Fix them.

In [32]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

Created <Metabolite pqq_c at 0x7fa1f4446450> in <ComplexFormation formation_GLUCDEHYDROG-MONOMER_mod_pqq at 0x7fa1f44463d0>
Created <Metabolite hemed_c at 0x7fa1f43db410> in <ComplexFormation formation_CYT-D-UBIOX-CPLX_mod_pheme_mod_hemed at 0x7fa1f43db390>
Created <Metabolite acetyl_c at 0x7fa1f43db7d0> in <ComplexFormation formation_rpL7/12_mod_1:acetyl at 0x7fa1f43db750>
Created <Metabolite EG50003-MONOMER_mod_pan4p at 0x7fa1f43f0510> in <ComplexFormation formation_GCVMULTI-CPLX_mod_lipo at 0x7fa1f43f0250>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_lipo at 0x7fa1f43f0590> in <ComplexFormation formation_GCVMULTI-CPLX_mod_lipo at 0x7fa1f43f0250>
Created <Metabolite 23bpg_c at 0x7fa1f43c03d0> in <ComplexFormation formation_PHOSGLYCMUTASE_mod_23bpg at 0x7fa1f43c0350>
Created <Metabolite dpm_c at 0x7fa1f437db90> in <ComplexFormation formation_OHMETHYLBILANESYN-MONOMER_mod_dpm at 0x7fa1f437db10>
Created <Metabolite tl_c at 0x7fa1f4326910> in <ComplexFormation formation_S-ADENMETSYN

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [33]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_modified_complex(generic=True)

Fixed _DASH:  D__LACtex
Fixed _DASH:  L__LACD2
Fixed _DASH:  L__LACD3
Fixed _DASH:  D__LACt2pp
Fixed _DASH:  L__LACtex
Fixed _DASH:  L__LACt2rpp


In [34]:
for reaction_data in me.stoichiometric_data:
    # Some reactions have no complex associated. Determine if they are
    # spontaneous or orphan reactions.
    try:
        complexes = rxnToModCplxDict[reaction_data.id]
    except KeyError:
        # These are orphans catalyzed by a dummy
        if reaction_data.id == "dummy_reaction" or \
                not reaction_info.is_spontaneous[reaction_data.id]:
            complexes = ["CPLX_dummy"]
        # These are truly spontaneous
        else:
            complexes = [None]
    for complex_id in complexes:
        complex_data = me.complex_data.get_by_id(complex_id) if complex_id else None
        if reaction_data.lower_bound < 0:
            r = MetabolicReaction(reaction_data.id + "_REV_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = True
            r.update(create_new=True)
        if reaction_data.upper_bound > 0:
            r = MetabolicReaction(reaction_data.id + "_FWD_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = False
            r.update(create_new=True)

Created <Metabolite CPLX0-7817_mod_Oxidized at 0x7fa1f4260210> in <MetabolicReaction RNDR1b3_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa1f42601d0>
Created <Metabolite GLUTAREDOXIN-MONOMER_mod_Oxidized at 0x7fa1f4260290> in <MetabolicReaction RNDR1b1_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa1f4260250>
Created <Metabolite GRXC-MONOMER_mod_Oxidized at 0x7fa1f4260310> in <MetabolicReaction RNDR1b4_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa1f42602d0>
Created <Metabolite GRXB-MONOMER_mod_Oxidized at 0x7fa1f4260490> in <MetabolicReaction RNDR1b2_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7fa1f4260450>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_3ha at 0x7fa1f4260a90> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7fa1f4260a50>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_act at 0x7fa1f4260ad0> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7fa1f4260a50>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_ddca at 0x7fa1f4260c50> in <Me

In [35]:
# This reaction is weird
me.reactions.get_by_id('CITLY-CPLX_2tpr3dpcoa_FWD_G6340-MONOMER').reaction

'4.27350427350427e-6*mu G6340-MONOMER + 2tpr3dpcoa_c --> '

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [36]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

In [37]:
for met in me.metabolites.query('RNA_b'):
    if met.RNA_type == 'mRNA' and len(me.reactions) > 1:
        print met.id

RNA_b0001
RNA_b0002
RNA_b0003
RNA_b0004
RNA_b0005
RNA_b0006
RNA_b0007
RNA_b0008
RNA_b0009
RNA_b0010
RNA_b0011
RNA_b0013
RNA_b0014
RNA_b0015
RNA_b0016
RNA_b0018
RNA_b4412
RNA_b0019
RNA_b0020
RNA_b0021
RNA_b0022
RNA_b0023
RNA_b0024
RNA_b0025
RNA_b0026
RNA_b0027
RNA_b0028
RNA_b0029
RNA_b0030
RNA_b0031
RNA_b0032
RNA_b0033
RNA_b0034
RNA_b0035
RNA_b0036
RNA_b0037
RNA_b0038
RNA_b0039
RNA_b0040
RNA_b0041
RNA_b0042
RNA_b0043
RNA_b0044
RNA_b0045
RNA_b0046
RNA_b0047
RNA_b0048
RNA_b0049
RNA_b0050
RNA_b0051
RNA_b0052
RNA_b0053
RNA_b0054
RNA_b0055
RNA_b4659
RNA_b0058
RNA_b0059
RNA_b0060
RNA_b0061
RNA_b0062
RNA_b0063
RNA_b0064
RNA_b0065
RNA_b0066
RNA_b0067
RNA_b0068
RNA_b0069
RNA_b4662
RNA_b0070
RNA_b0071
RNA_b0072
RNA_b0073
RNA_b0074
RNA_b0075
RNA_b0076
RNA_b0077
RNA_b0078
RNA_b0080
RNA_b0081
RNA_b0082
RNA_b0083
RNA_b0084
RNA_b0085
RNA_b0086
RNA_b0087
RNA_b0088
RNA_b0089
RNA_b0090
RNA_b0091
RNA_b0092
RNA_b0093
RNA_b0094
RNA_b0095
RNA_b0096
RNA_b0097
RNA_b0098
RNA_b0099
RNA_b0101
RNA_b0102
RNA_b0103


In [38]:
me.metabolites.RNA_b0207.reactions

frozenset({<Reaction DM_b0207 at 0x7fa1fb272f10>,
           <TranscriptionReaction transcription_TU0_12839_from_RPOS_MONOMER at 0x7fa1f7c75150>,
           <TranslationReaction translation_b0207 at 0x7fa1fb272ed0>})

## Add in translocation

In [39]:
translocPath = pandas.read_csv(join(ecoli_files_dir, "translocation_pathways.txt"),sep='\t')

for index, row in translocPath.iterrows():
    translocRxn = ProteinTranslocationData(row.Reaction_name, me)
    translocRxn.keff = float(row.Keff)
    translocRxn.costs_complexes = row.Complexes.split(' AND ')

In [40]:
proteins_sa_coeff_inner={}

transloc = pandas.read_csv(join(ecoli_files_dir, "peptide_compartment_and_pathways2.txt"), sep='\t', comment="#")
for index, row in transloc.iterrows():
    me.metabolites.get_by_id(row.Complex).compartment = row.Complex_compartment
    me.metabolites.get_by_id('protein_'+row.Protein.split('(')[0]).compartment = row.Protein_compartment
    #if index > 316:
    #    continue
    if row.translocase_pathway=='s':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        
        ## This is in preparation for membrane constraint
        mass = me.translation_data.get_by_id(row.Protein.split('(')[0]).mass
        if row.Complex in proteins_sa_coeff_inner.keys():
            proteins_sa_coeff_inner[row.Complex]+=mass*1.21/42.*2
        else:
            proteins_sa_coeff_inner[row.Complex]=mass*1.21/42.*2
    elif row.translocase_pathway == 'r':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)       
    elif row.translocase_pathway == 'p':
        me.translocation_pathways.srp_yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)         
    elif row.translocase_pathway =='t':
        me.translocation_pathways.tat_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway=='a':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        me.translocation_pathways.secA_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='l':
        me.translocation_pathways.lol_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='b':
        me.translocation_pathways.bam_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='y':
        me.translocation_pathways.yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)        
    elif row.translocase_pathway!='n':
        print row.translocase_pathway        

sb
pla
sb
sb
sb
pa
sb
sb
pla
pla
sb
sb
sb
pla
pa
sb
sb
sb
pa
sb
sb
sb
sb
sb
sb
sb
pla
pa
sb
sb
sb
pla
sb
sb
pla
sb
pla
pla
pla
pla
sb
pla
ra


Remove unused protein and mRNA to make the model solve faster (TODO remove unused complexes too)

In [41]:
one_rxn_list = []
for met in me.metabolites:
    if len(me.metabolites.get_by_id(met.id.rstrip('_')).reactions) == 1:
        print met.id
        one_rxn_list.append(met.id)

cpg180_c
selnp_c
ragund_c
23doguln_c
4mhetz_c
4hthr_c
cpe160_c
gdpfuc_c
cinnm_c
preq1_c
hemeO_c
aconm_c
didp_c
hqn_c
malt6p_c
thmnp_c
n8aspmd_c
dxyl_c
mercppyr_c
lipoamp_c
mi1p__D_c
mococdp_c
met__D_c
2pglyc_c
acanth_c
xdp_c
na15dap_c
cpg160_c
sarcs_c
cpe180_c
cs_e
athtp_c
ap5a_c
aso4_c
dmbzid_c
Nmtrp_c
gp4g_c
ag_c
N1aspmd_c
tungs_c
RNA_b4659
RNA_b4587
RNA_b4690
RNA_b0258
RNA_b0263
RNA_b4505
RNA_b0266
RNA_b4629
RNA_b4694
RNA_b4695
RNA_b0499
RNA_b4572
RNA_b0561
RNA_b4635
RNA_b4514
RNA_b4417
RNA_b4692
RNA_b4693
RNA_b1151
RNA_b1152
RNA_b1157
RNA_b4521
RNA_b4524
RNA_b4525
RNA_b4699
RNA_b1361
RNA_b1362
RNA_b1368
RNA_b1370
RNA_b1471
RNA_b1472
RNA_b1548
RNA_b1567
RNA_b1568
RNA_b1577
RNA_b1578
RNA_b1579
RNA_b4431
RNA_b2006
RNA_b4538
RNA_b4436
RNA_b4437
RNA_b4500
RNA_b4661
RNA_b2353
RNA_b2355
RNA_b2356
RNA_b4545
RNA_b4643
RNA_b2638
RNA_b2681
RNA_b4442
RNA_b4701
RNA_b4443
RNA_b2856
RNA_b2858
RNA_b2859
RNA_b4446
RNA_b2981
RNA_b3134
RNA_b3135
RNA_b3268
RNA_b3427
RNA_b4704
RNA_b4454
RNA_b4614
RNA_b

In [42]:
me.reactions.L_DASH_LACD2_FWD_CPLX_dummy.reaction

'4.27350427350427e-6*mu CPLX_dummy + q8_c + lac__L_c --> q8h2_c + pyr_c'

In [43]:
import cobra.test
ijo = cobra.test.create_test_model(cobra.test.ecoli_pickle)

In [44]:
for met in ijo.metabolites:
    if len(met.reactions) == 1:
        print met.id

23doguln_c
2pglyc_c
2tpr3dpcoa_c
3sala_c
4ahmmp_c
4hthr_c
4mhetz_c
56dura_c
N1aspmd_c
Nmtrp_c
acanth_c
acglc__D_c
acmalt_c
aconm_c
ag_c
alatrna_c
ap5a_c
apoACP_c
appl_c
argtrna_c
asntrna_c
aso4_c
asptrna_c
athtp_c
bbtcoa_c
bwcogdp_c
cinnm_c
cpe160_c
cpe180_c
cpg160_c
cpg180_c
cystrna_c
dhmptp_c
didp_c
dmbzid_c
dxyl_c
fmettrna_c
gdpfuc_c
ghb_c
glntrna_c
glytrna_c
gp4g_c
histrna_c
hqn_c
iletrna_c
leutrna_c
lystrna_c
malt6p_c
mercppyr_c
met__D_c
mi1p__D_c
n8aspmd_c
na15dap_c
oxa_c
phetrna_c
preq1_c
protrna_c
ragund_c
sarcs_c
sectrna_c
sertrna_c
thmnp_c
thrtrna_c
trnaala_c
trnaarg_c
trnaasn_c
trnaasp_c
trnacys_c
trnagln_c
trnagly_c
trnahis_c
trnaile_c
trnaleu_c
trnalys_c
trnamet_c
trnaphe_c
trnapro_c
trnasecys_c
trnaser_c
trnathr_c
trnatrp_c
trnatyr_c
trnaval_c
trptrna_c
tyrtrna_c
valtrna_c
xdp_c


In [45]:
me.metabolites.protein_b0245.reactions

frozenset({<TranslationReaction translation_b0245 at 0x7fa1fb4e5250>})

In [46]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)
for p in me.metabolites.query("protein"):
    if len(p._reaction) == 1:
        list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    if len(m._reaction) == 1:
        list(m._reaction)[0].delete(remove_orphans=True)

This gives the total number of genes included

In [47]:
len(me.reactions.query("transcription"))

5214

## Attempt to set keffs

In [48]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i, 'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

no keff found for keff_METAT_S_DASH_ADENMETSYN_DASH_CPLX_mod_4_COLON_generic_monovalent_mod_1_COLON_generic_divalent
no keff found for keff_GLUTRR_CPLX0_DASH_3741


## Solve

In [49]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [50]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction+'_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [51]:
me.reactions.EX_glc__D_e.lower_bound = -100

In [52]:
expressions = compile_expressions(me)

In [53]:
solve_at_growth_rate(me, 0.1, compiled_expressions=expressions)

<Solution 'infeasible' at 0x7fa1ede8fd50>

In [ ]:
binary_search(me, min_mu=0, max_mu=.0001, mu_accuracy=1e-3,
              compiled_expressions=expressions)

LP files will be saved in /tmp/tmpUda60s
mu	status	basis	time	iter	obj
0.0000	+	False	0.47	128	1000.0

In [ ]:
me.compute_solution_error()

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()